# Darcy equation

In this tutorial we present how to solve an evolutionary Darcy equation with [PyGeoN](https://github.com/compgeo-mox/pygeon).  The unkwons are the velocity $q$ and the pressure $p$.

Let $\Omega=(0,1)^2$ with boundary $\partial \Omega$ and outward unit normal ${\nu}$. Let $(0,T)$ with $10=T>0$ be the overall simulation period. Given 
$k$ the matrix permeability, we want to solve the following problem: find $({q}, p)$ such that
$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
k^{-1} {q} + \nabla p = {- \rho g \nabla y}\\
p_t + \nabla \cdot {q} = f
\end{array}
&\text{in } \Omega \times (0,T)
\end{array}
\right.
$$
with boundary conditions:
$$ p = 0 \text{ on } \partial_{top} \Omega \times (0,T] \qquad p = \rho g \text{ on } \partial_{bottom} \Omega \times (0,T] \qquad \nu \cdot q = 0 \text{ on } \partial_{left} \Omega \cup \partial_{right} \Omega \times (0,T] $$
and initial conditions:
$$ p|_{t=0} = (1-y) \rho g \text{ in } \Omega \qquad q|_{t=0} = 0 \text{ in } \Omega $$

We present *step-by-step* how to create the grid, declare the problem data, and finally solve the problem.

First we import some of the standard modules, like `numpy` and `scipy.sparse`. Since PyGeoN is based on [PorePy](https://github.com/pmgbergen/porepy) we import both modules.

In [1]:
%load_ext Cython

In [2]:
import shutil
import os

import numpy as np
import scipy.sparse as sps

import porepy as pp
import pygeon as pg

from math import ceil

/opt/conda/lib/python3.11/site-packages/porepy/numerics/nonlinear/nonlinear_solvers.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange  # type: ignore


### Parameters

In [3]:
N = 10
dt = 0.1

T = 3
S_s = 0.1

output_directory = 'hard_coupling'

### Mesh

In [4]:
bottom_grid =   pp.StructuredTriangleGrid([N, N], [1, 1])
bottom_grid.compute_geometry()

In [5]:
top_grid    = pp.StructuredTriangleGrid([N, 2*N], [1, 1])
top_grid.compute_geometry()

In [6]:
boundary_grid, boundary_face_map, boundary_node_map  = pp.partition.extract_subgrid(bottom_grid, 
                                                                                    bottom_grid.face_centers[1, :] == 1, 
                                                                                    faces=True)

### Discretization

In [7]:
key = "flow"

In [8]:
top_field = pg.Lagrange1(key)
top_data = {}

top_dof =  top_field.ndof( top_grid )

In [9]:
bottom_field = pg.Lagrange1(key)
bottom_data = {}

bottom_dof = bottom_field.ndof( bottom_grid )

In [10]:
boundary_field = pg.Lagrange1(key)
boundary_data = {}

boundary_dof = boundary_field.ndof( boundary_grid )

### Restrictor Matrices

In [11]:
bottom_delete_boundary_dof = np.zeros( shape=(bottom_dof - boundary_dof, bottom_dof) )
bottom_delete_boundary_dof[np.arange(bottom_dof - boundary_dof), bottom_grid.nodes[1, :] < 1] = 1
bottom_delete_boundary_dof = sps.csr_matrix(bottom_delete_boundary_dof)

In [12]:
bottom_isolate_boundary_dof = np.zeros(shape=(boundary_dof, bottom_dof))
bottom_isolate_boundary_dof[np.arange(boundary_dof), bottom_grid.nodes[1, :] == 1] = 1
bottom_isolate_boundary_dof = sps.csr_matrix(bottom_isolate_boundary_dof)

In [13]:
top_delete_boundary_dof = np.zeros( shape=(top_dof - boundary_dof, top_dof) )
top_delete_boundary_dof[np.arange(top_dof - boundary_dof), top_grid.nodes[1, :] > 0] = 1
top_delete_boundary_dof = sps.csr_matrix(top_delete_boundary_dof)

In [14]:
top_isolate_boundary_dof = np.zeros(shape=(boundary_dof, top_dof))
top_isolate_boundary_dof[np.arange(boundary_dof), top_grid.nodes[1, :] == 0] = 1
top_isolate_boundary_dof = sps.csr_matrix(top_isolate_boundary_dof)

### BCs

In [15]:
bot_dirichlet_flag  = np.array(bottom_delete_boundary_dof @ (bottom_grid.nodes[0, :] == 1), dtype=bool)
bot_dirichlet_value = np.array(bot_dirichlet_flag, dtype=float)

In [16]:
top_dirichlet_flag  = np.array(top_delete_boundary_dof @ (top_grid.nodes[1, :] == 1), dtype=bool)
top_dirichlet_value = np.array(top_dirichlet_flag, dtype=float) * 2

In [17]:
dirichlet_flag  = np.hstack(( bot_dirichlet_flag, np.zeros(shape=boundary_dof, dtype=bool), top_dirichlet_flag ))
dirichlet_value = np.hstack(( bot_dirichlet_value, np.zeros(shape=boundary_dof), top_dirichlet_value))

### Matrix Assembly

##### Mass matrices

In [18]:
top_mass = top_field.assemble_mass_matrix( top_grid )

In [19]:
bottom_mass = bottom_field.assemble_mass_matrix(bottom_grid)

##### Projection matrices

In [20]:
top_proj    = top_field.eval_at_cell_centers( top_grid )
bottom_proj = bottom_field.eval_at_cell_centers( bottom_grid )

##### Stifness

In [21]:
%%cython
import numpy as np

def find_ordering(coord: np.array):
    lx = np.argmin(coord[0, :])
    rx = np.argmax(coord[0, :])
    mx = np.setdiff1d(np.array([0,1,2]), np.array([lx, rx]))[0]

    # Vertical Alignment
    if np.abs( coord[0, lx] - coord[0, mx] ) < 1e-7:
        # lx and mx vertical aligned, rx no
        up =   lx if np.argmax(coord[1, np.array([lx, mx])]) == 0 else mx
        down = lx if np.argmin(coord[1, np.array([lx, mx])]) == 0 else mx

        if np.abs( coord[1, up] - coord[1, rx] ) < 1e-7:
            return [up, down, rx]
        else:
            return [down, rx, up]
    else:
        # rx and mx vertical aligned, lx no
        up =   rx if np.argmax(coord[1, np.array([rx, mx])]) == 0 else mx
        down = rx if np.argmin(coord[1, np.array([rx, mx])]) == 0 else mx

        if np.abs( coord[1, up] - coord[1, lx] ) < 1e-7:
            return [up, lx, down]
        else:
            return [down, up, lx]

In [22]:
%%cython
import numpy as np

def q1():
    return np.array([-1, -1])

def q2():
    return np.array([ 1, 0])

def q3():
    return np.array([0,  1])

In [23]:
def local_A(coord):

    element_height = (np.max(coord[1, :]) - np.min(coord[1, :]))
    element_width  = (np.max(coord[0, :]) - np.min(coord[0, :]))

    ordering = find_ordering(coord)

    x0 = coord[:, ordering][:, 0]
    x1 = coord[:, ordering][:, 1]
    x2 = coord[:, ordering][:, 2]
    
    J_T_1_T = np.array([[x2[1]-x0[1], x0[1]-x1[1]],
                        [x0[0]-x2[0], x1[0]-x0[0]]]) / ((x1[0]-x0[0]) * (x2[1]-x0[1]) - (x2[0]-x0[0]) * (x1[1]-x0[1]))
    

    q_funcs = [J_T_1_T @ q1(), J_T_1_T @ q2(), J_T_1_T @ q3()]

    M = np.zeros(shape=(3,3))

    area = element_height * element_width

    for i in range(3):
        for j in range(3):
            M[ ordering[i], ordering[j] ] = q_funcs[j].T @ np.eye(2) @ q_funcs[i] * area

    return M

In [24]:
def stifness(subdomain):

    # Map the domain to a reference geometry (i.e. equivalent to compute
    # surface coordinates in 1d and 2d)

    _, _, _, _, _, node_coords = pp.map_geometry.map_grid(subdomain)

    # Allocate the data to store matrix entries, that's the most efficient
    # way to create a sparse matrix.
    size = np.power(subdomain.dim + 1, 2) * subdomain.num_cells
    rows_I = np.empty(size, dtype=int)
    cols_J = np.empty(size, dtype=int)
    data_IJ = np.empty(size)
    idx = 0

    cell_nodes = subdomain.cell_nodes()

    for c in np.arange(subdomain.num_cells):
        # For the current cell retrieve its nodes
        loc = slice(cell_nodes.indptr[c], cell_nodes.indptr[c + 1])

        nodes_loc = cell_nodes.indices[loc]
        coord_loc = node_coords[:, nodes_loc]

        # Compute the stiff-H1 local matrix
        A = local_A(coord_loc)

        # Save values for stiff-H1 local matrix in the global structure
        cols = np.tile(nodes_loc, (nodes_loc.size, 1))
        loc_idx = slice(idx, idx + cols.size)
        rows_I[loc_idx] = cols.T.ravel()
        cols_J[loc_idx] = cols.ravel()
        data_IJ[loc_idx] = A.ravel()
        idx += cols.size

    # Construct the global matrices
    return sps.csc_matrix((data_IJ, (rows_I, cols_J)))

### Solving stage

In [25]:
# assemble initial solution
initial_solution = np.zeros(top_dof + bottom_dof - boundary_dof)

In [26]:
bottom_mask = np.zeros_like(initial_solution, dtype=bool)
bottom_mask[ : (bottom_dof) ] = True

internal_bottom_mask = np.zeros_like(bottom_mask, dtype=bool)
internal_bottom_mask[ : (bottom_dof - boundary_dof)] = True

top_mask = np.zeros_like(initial_solution, dtype=bool)
top_mask[(bottom_dof - boundary_dof) : ] = True

internal_top_mask = np.zeros_like(bottom_mask, dtype=bool)
internal_top_mask[ bottom_dof : ] = True

boundary_mask = np.zeros_like(bottom_mask, dtype=bool)
boundary_mask[(bottom_dof - boundary_dof) : bottom_dof] = True

In [27]:
initial_solution[top_mask]    =    top_field.interpolate(   top_grid, lambda x: 1)
initial_solution[bottom_mask] = bottom_field.interpolate(bottom_grid, lambda x: 1)

In [28]:
top_saver    = pp.Exporter(top_grid,    "top_sol", folder_name=output_directory)
bottom_saver = pp.Exporter(bottom_grid, "bottom_sol", folder_name=output_directory)

def save_step(current_sol, step):
    ins = list()

    ins.append((top_grid, "cell_h", top_proj @ current_sol[top_mask]))
    top_saver.write_vtu(ins, time_step=step)

    ins = list()

    ins.append((bottom_grid, "cell_h", bottom_proj @ current_sol[bottom_mask]))
    bottom_saver.write_vtu(ins, time_step=step)

In [29]:
if os.path.exists(output_directory):
    shutil.rmtree(output_directory)

In [30]:
# solve the problem

sol = [initial_solution]

t = 0

save_step(sol[-1], 0)

We need to solve the linear system, PyGeoN provides a framework for that. The actual imposition of essential boundary conditions (flux boundary conditions) might change the symmetry of the global system, the class `pg.LinearSystem` preserves this structure by internally eliminating these degrees of freedom. Once the problem is solved, we extract the two solutions $q$ and $p$.

In [31]:
bottom_stifness = stifness(bottom_grid)
top_stifness    = stifness(top_grid)

# Time Loop
for step in range(1, ceil(T/dt) + 1):
    current_time = step * dt
    
    rhs = np.zeros_like(sol[-1])

    rhs[internal_bottom_mask] += S_s / dt * bottom_delete_boundary_dof @ bottom_mass @ sol[-1][bottom_mask]
    rhs[internal_top_mask]    += S_s / dt * top_delete_boundary_dof    @ top_mass    @ sol[-1][top_mask]
    rhs[boundary_mask]        += S_s / dt * (bottom_isolate_boundary_dof @ bottom_mass @ sol[-1][bottom_mask] + top_isolate_boundary_dof @ top_mass @ sol[-1][top_mask])

    bottom_mat = S_s / dt * bottom_mass + bottom_stifness
    top_mat    = S_s / dt * top_mass + top_stifness
    mid_mat = top_isolate_boundary_dof @ top_mat @ top_isolate_boundary_dof.T + bottom_isolate_boundary_dof @ bottom_mat @ bottom_isolate_boundary_dof.T

    spp = sps.bmat([[    bottom_delete_boundary_dof @ bottom_mat @ bottom_delete_boundary_dof.T, bottom_delete_boundary_dof @ bottom_mat @ bottom_isolate_boundary_dof.T,                                                             None],
                    [ bottom_isolate_boundary_dof @ bottom_mat.T @ bottom_delete_boundary_dof.T,                                                                 mid_mat, top_isolate_boundary_dof @ top_mat.T @ top_delete_boundary_dof.T],
                    [                                                                      None,          top_delete_boundary_dof @ top_mat @ top_isolate_boundary_dof.T,    top_delete_boundary_dof @ top_mat @ top_delete_boundary_dof.T]], format = 'csc')
     
    ls = pg.LinearSystem(spp, rhs)
    ls.flag_ess_bc(dirichlet_flag, dirichlet_value)

    sol.append( ls.solve() )

    save_step(sol[-1], step)